In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import tensorflow as tf

SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

In [ ]:
csv_file = '/kaggle/input/tabular-playground-series-jan-2022/train.csv'

In [ ]:
test_csv_file = '/kaggle/input/tabular-playground-series-jan-2022/test.csv'

In [ ]:
df = pd.read_csv(csv_file)

In [ ]:
test_df = pd.read_csv(test_csv_file)

In [ ]:
df.head()

In [ ]:
test_df.head()

In [ ]:
df.dtypes

In [ ]:
target = df.pop('num_sold)

In [ ]:
df.pop('row_id')
df.pop('date')

In [ ]:
test_df.pop('row_id')
test_df.pop('date')

In [ ]:
numeric_feature_names = []
numeric_features = df[numeric_feature_names]
numeric_features.head()

In [ ]:
tf.convert_to_tensor(numeric_features)

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

In [ ]:
normalizer(numeric_features.iloc[:3])

In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
binary_feature_names = []

In [ ]:
categorical_feature_names = ['country', 'store', 'product']

In [ ]:
inputs = {}
for name, column in df.items():
  if type(column[0]) == str:
    dtype = tf.string
  elif (name in categorical_feature_names or
        name in binary_feature_names):
    dtype = tf.int64
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)

In [ ]:
test_inputs = {}
for name, column in test_df.items():
  if type(column[0]) == str:
    dtype = tf.string
  elif (name in categorical_feature_names or
        name in binary_feature_names):
    dtype = tf.int64
  else:
    dtype = tf.float32

  test_inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)

In [ ]:
inputs

In [ ]:
test_inputs

In [ ]:
preprocessed = []

for name in binary_feature_names:
  inp = inputs[name]
  inp = inp[:, tf.newaxis]
  float_value = tf.cast(inp, tf.float32)
  preprocessed.append(float_value)

preprocessed

In [ ]:
test_preprocessed = []

test_preprocessed

In [ ]:
for name in categorical_feature_names:
  vocab = sorted(set(df[name]))
  print(f'name: {name}')
  print(f'vocab: {vocab}\n')

  if type(vocab[0]) is str:
    lookup = tf.keras.layers.StringLookup(vocabulary=vocab, output_mode='one_hot')
  else:
    lookup = tf.keras.layers.IntegerLookup(vocabulary=vocab, output_mode='one_hot')

  x = inputs[name][:, tf.newaxis]
  x = lookup(x)
  preprocessed.append(x)

  test_x = test_inputs[name][:, tf.newaxis]
  test_x = lookup(test_x)
  test_preprocessed.append(test_x)

In [ ]:
preprocessed

In [ ]:
test_preprocessed

In [ ]:
preprocesssed_result = tf.concat(preprocessed, axis=-1)
preprocesssed_result

In [ ]:
test_preprocesssed_result = tf.concat(test_preprocessed, axis=-1)
test_preprocesssed_result

In [ ]:
preprocessor = tf.keras.Model(inputs, preprocesssed_result)

In [ ]:
tf.keras.utils.plot_model(preprocessor, rankdir="LR", show_shapes=True)

In [ ]:
preprocessor(dict(df.iloc[:1]))

In [ ]:
body = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [ ]:
inputs

In [ ]:
x = preprocessor(inputs)
x

In [ ]:
test_x = preprocessor(test_inputs)
test_x

In [ ]:
result = body(x)
result

In [ ]:
model = tf.keras.Model(inputs, result)

model.compile(optimizer='rmsprop',
                loss="mse",
                metrics=['mae'])

In [ ]:
history = model.fit(dict(df), target, epochs=5, batch_size=BATCH_SIZE)

In [ ]:
predictions = model.predict(test_x)